# 3D Transforms

In this we demonstrate 3D WarpAffine and Rotate.

### Warp operators
All warp operators work by caclulating the output pixels by sampling the source image at transformed coordinates:

$${Out}(x, y, z) = {In}(x_{src}, y_{src}, z_{src})$$

This way each output pixel is calculated exactly once.

If the source coordinates do not point exactly to pixel centers, the values of neighboring pixels will be interpolated or the nearst pixel is taken, depending on the interpolation method specified in the `interp_type` argument.

### Affine transform

The source sample coordinates $x_{src}, y_{src}$ are calculated according to the formula:

$$
\begin{vmatrix}
x_{src} \\
y_{src}
\end{vmatrix}
= \begin{vmatrix}
m_{00} & m_{01} & m_{02} & t_x \\
m_{10} & m_{11} & m_{12} & t_y \\
m_{20} & m_{21} & m_{22} & t_z
\end{vmatrix}
\begin{vmatrix}
{x} \\
{y} \\
{z} \\
1
\end{vmatrix}
$$

Where $x, y$ are coordinates of the destination pixel and the matrix represents the inverse (destination to source) affine transform. The 
$\begin{vmatrix}
m_{00} & m_{01} & m_{02} \\
m_{10} & m_{11} & m_{12} \\
m_{20} & m_{21} & m_{22}
\end{vmatrix}$ block represents a combined rotate/scale/shear transform and $t_x, t_y, t_z$ is a translation vector.

### Rotation

Rotate is implemented in terms of affine transform, but calculates the transform matrix internally. The output size is automatically adjusted and the size parity is adjusted to reduce blur near the volume centre. The rotation matrix is inversed so that the transform appears to work "forward" (source to destination). The rotation is defined by specifying axis (as a vector) and angle (in degrees).

This (forward) transform is calculated as:

$$
\begin{vmatrix}
u^2 + (v^2 + w^2) \cdot \cos \alpha &
uv\cdot(1-\cos \alpha) - w\cdot \sin \alpha &
uw\cdot(1-\cos \alpha) + v\cdot \sin \alpha \\
uv\cdot(1-\cos \alpha) - w\cdot \sin \alpha &
v^2 + (u^2 + w^2) \cdot \cos \alpha &
vw\cdot(1-\cos \alpha) - u\cdot \sin \alpha \\
uw\cdot(1-\cos \alpha) - v\cdot \sin \alpha &
vw\cdot(1-\cos \alpha + u\cdot \sin \alpha &
w^2 + (u^2 + w^2) \cdot \cos \alpha
\end{vmatrix}
$$
where $u, v, w$ is normalized `axis` vector and $\alpha$ is `angle` converted from degrees to radians.


In [1]:
from __future__ import division
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types
import numpy as np
import matplotlib.pyplot as plt
import math
import os.path

ImportError: /home/michalz/Projects/DALI_build/release/dali/python/nvidia/dali/libdali_operators.so: version `DALI_OPERATORS_0.16.0dev' not found (required by /home/michalz/.local/lib/python3.6/site-packages/nvidia/dali/backend_impl.cpython-36m-x86_64-linux-gnu.so)

Let's define some test data - in this case, it consists of edges of a cube

In [ ]:
nvidia_green = [0x76,0xb9,00]
violet = [0xff - 0x76, 0xff - 0xb9, 0xff]

def lerp(a, b, w):
    return a*(1-w)+b*w

def build_cube(output_shape, size, color_front = nvidia_green, color_back = violet):
    color_front = np.array(color_front)
    color_back = np.array(color_back)
    arr = np.zeros(output_shape + [3], np.uint8)
    lo = (np.array(output_shape) - np.array(size)) * 0.5
    hi = (np.array(output_shape) + np.array(size)) * 0.5
    dims = len(output_shape)
    for d in range(dims):
        for c in range(int(np.rint(lo[d])), int(np.rint(hi[d]))):
            d0 = 0
            d1 = 1
            if d == 0:
                d0 = 1
                d1 = 2
            elif d == 1:
                d0 = 0
                d1 = 2
            p = [int(x) for x in lo]
            def color():
                return lerp(color_front, color_back, (p[0]-lo[0])/(hi[0]-lo[0]))

            p[d] = c
            arr[p[0], p[1], p[2], :] = color()
            p[d0] = int(np.rint(hi[d0]))
            arr[p[0], p[1], p[2], :] = color()
            p[d1] = int(np.rint(hi[d1]))
            arr[p[0], p[1], p[2], :] = color()
            p[d0] = int(np.rint(lo[d0]))
            arr[p[0], p[1], p[2], :] = color()
    return arr

Now, let's define the pipeline.

In [ ]:
class ExamplePipeline(Pipeline):
    def __init__(self, batch_size, num_threads, device_id, pipelined = True, exec_async = True):
        super(ExamplePipeline, self).__init__(
            batch_size, num_threads, device_id,
            seed = 12, exec_pipelined=pipelined, exec_async=exec_async)
        
        self.input = ops.ExternalSource()
       
        self.rotate_gpu = ops.Rotate(
            device = "gpu",
            axis = (0.5,1,1),
            angle = 30,
            interp_type = types.INTERP_LINEAR # use linear interpolation
        )
        self.rotate_cpu = ops.Rotate(
            device = "cpu",
            axis = (0.5,1,1),
            angle = 30,
            interp_type = types.INTERP_LINEAR # use linear interpolation
        )
        
    # Then, we can tie the operators together to form a graph
        
    def define_graph(self):
        self.data = self.input()
        outputs = [self.data.gpu()]
        # pass the transform parameters through GPU memory
        outputs += [self.rotate_gpu(self.data.gpu()), self.rotate_cpu(self.data).gpu()]

        return outputs
    
    # Since we're using ExternalSource, we need to feed the externally provided data to the pipeline
    
    def iter_setup(self):
        # Generate the transforms for the batch and feed them to the ExternalSource
        self.feed_input(self.data, [build_cube([128,128,128],[80,80,80]), build_cube([160,160,160],[80,120,80])])    

The pipeline class is now ready to use - we need to construct and build it before we run it.

In [ ]:
batch_size = 2
pipe = ExamplePipeline(batch_size=batch_size, num_threads=2, device_id = 0)
pipe.build()

Finally, we can call `run` on our pipeline to obtain the first batch of preprocessed images.

In [ ]:
pipe_out = pipe.run()

## Example output

Now that we've processed the first batch of images, let's see the results. The output is projected by taking maximum intensity values along Z axis.

In [ ]:
n = 1  # change this value to see other images from the batch;
       # it must be in 0..batch_size-1 range

def project_volume(volume):
    return np.amax(volume, axis = 0)    

from synsets import imagenet_synsets
import matplotlib.gridspec as gridspec

len_outputs = len(pipe_out)

captions = ["original",
            "rotated (gpu)", "rotated (cpu)"]

fig = plt.figure(figsize = (16,12))
plt.suptitle("3D rotate", fontsize=16)
columns = 2
rows = int(math.ceil(len_outputs / columns))
gs = gridspec.GridSpec(rows, columns)

for i in range(len_outputs):
    plt.subplot(gs[i])
    plt.axis("off")
    plt.title(captions[i])
    pipe_out_cpu = pipe_out[i].as_cpu()
    img = project_volume(pipe_out_cpu.at(n)) * (1 / 255.0)
    plt.imshow(img)